In [ ]:
!pip install -r requirements_2.txt
#!pip install matplotlib

In [ ]:
# just setting up functions to do demand stuff with 

import numpy as np
import pandas as pd
from cfe.estimation import drop_columns_wo_covariance
from cfe import Regression
from eep153_tools.sheets import read_sheets
!pip install fooddatacentral

In [ ]:
sheets = '1J7d0WCnpAcMoaAScLzqaAEeexuUPdcPRYO42OPjJPu8' #read in sheets

In [ ]:
x = read_sheets(sheets,sheet='Food Expenditures') # grab food expenditures, set meta index
x.columns.name = 'j'

p = read_sheets(sheets,sheet='PriceAsColumns').set_index(['t','m']) # grab prices 
p.columns.name = 'j'

d = read_sheets(sheets,sheet="Household Characteristics") # grab household characteristics
d.columns.name = 'k'
d.set_index(['i','t','m'],inplace=True)

d_dummy = read_sheets(sheets, sheet='HHChar_wDummy')
d_dummy.columns.name = 'k'
d_dummy.set_index(['i', 't', 'm'], inplace=True)

x = x.T.groupby('j').sum().T # get rid of duplicates 
x = x.replace(0,np.nan) # make zeros into NaN

y = np.log(x.set_index(['i','t','m'])) # take log of expenditures, name y

In [ ]:
d_dummy['total'] = np.sum(d_dummy[['Males 00-03', 'Females 00-03', 'Males 04-08', 
                                'Females 04-08', 'Males 09-13', 'Females 09-13', 'Males 14-18',
                                'Females 14-18', 'Males 19-30', 'Females 19-30', 'Males 31-50',
                                'Females 31-50', 'Males 51-99', 'Females 51-99']], axis = 1)



In [ ]:
d_dummy

In [ ]:
# doing this to get index

d_R = d_dummy[d_dummy['Rural'] == 1]
d_U = d_dummy[d_dummy['Rural'] == 0]
#d_R.reset_index(inplace=True)
#d_U.reset_index(inplace=True)
d_R_bigHH = d_R[d_R['total']>8]
d_R_smallHH = d_R[d_R['total']<8]

d_U_bigHH = d_U[d_U['total']>8]
d_U_smallHH = d_U[d_U['total']<8]

In [ ]:
# grab indices from the segmented sets 

d_R_bigHH.reset_index(inplace=True)
rural_bigHH_index = d_R_bigHH['i']

d_R_smallHH.reset_index(inplace=True)
rural_smallHH_index = d_R_smallHH['i']

d_U_bigHH.reset_index(inplace=True)
urban_bigHH_index = d_U_bigHH['i']

d_U_smallHH.reset_index(inplace=True)
urban_smallHH_index = d_U_smallHH['i']

In [ ]:
# pull expenditures for these subsets

x_R_big = x[x['i'].isin(rural_bigHH_index)]
x_R_small = x[x['i'].isin(rural_smallHH_index)]

x_U_big = x[x['i'].isin(urban_bigHH_index)]
x_U_small = x[x['i'].isin(urban_smallHH_index)]

In [ ]:
y_R_big = np.log(x_R_big.set_index(['i','t','m'])) # take log of expenditures, name y
y_R_small = np.log(x_R_small.set_index(['i','t','m']))

y_U_big = np.log(x_U_big.set_index(['i','t','m'])) 
y_U_small = np.log(x_U_small.set_index(['i','t','m']))

In [ ]:
x_U_big['Maize']

In [ ]:
x_U_small['Maize']

In [ ]:
d_R_bigHH = d_R[d_R['total']>8]
d_R_smallHH = d_R[d_R['total']<8]

d_U_bigHH = d_U[d_U['total']>8]
d_U_smallHH = d_U[d_U['total']<8]

In [ ]:
d_R_big = d_R_bigHH.drop(columns='total')
d_R_small = d_R_smallHH.drop(columns='total')

d_U_big = d_U_bigHH.drop(columns='total')
d_U_small = d_U_smallHH.drop(columns='total')

In [ ]:
print(y_U_big.index.names)
print(d_R.index.names)

In [ ]:
from cfe import Regression

In [ ]:
# build function to run regression

def regress(y, d):
    
    y = y.stack()
    d = d.stack()
    # Check that indices are in right places!
    assert y.index.names == ['i','t','m','j']
    assert d.index.names == ['i','t','m','k']
    result = Regression(y=y,d=d)
    #predicted_expenditures = pd.DataFrame({'y':y,'yhat':result.get_predicted_log_expenditures()})
    return result

In [ ]:
# apply regression to household subsets and geographies

rural_smallHH = regress(y_R_small, d_R_small)
rural_bigHH = regress(y_R_big, d_R_big)

urban_smallHH = regress(y_U_small, d_U_small)
urban_bigHH = regress(y_U_big, d_U_big)

In [ ]:
#urban_smallHH.graph_beta()
#rural_bigHH.graph_beta()

In [ ]:
# fig, ax = plt.subplots(1,2, figsize = (10,5))

# ax[0].scatter(x_pred_R['yhat'], x_pred_R['y'])
# ax[0].set_xlabel('Actual Log Expenditures')
# ax[0].set_ylabel('Predicted Log Expenditures')
# ax[0].set_title('RURAL: Predicted vs. Actual Expenditures')

# ax[1].scatter(x_pred_U['yhat'], x_pred_U['y'])
# ax[1].set_xlabel('Actual Log Expenditures')
# ax[1].set_ylabel('Predicted Log Expenditures')
# ax[1].set_title('URBAN: Predicted vs. Actual Expenditures')

In [ ]:
rural_smallHH.get_beta()

In [ ]:
urban_smallHH.get_beta()

In [ ]:
beta_dict = {'Urban, Small HH' : urban_smallHH.get_beta(), 'Urban, Large HH': urban_bigHH.get_beta(), 
             'Rural, Small HH' : rural_smallHH.get_beta(), 'Rural, Large HH' : rural_bigHH.get_beta()}


betas = pd.DataFrame(beta_dict) # made a dataframe with Frisch elasticities for each good in each place

In [ ]:
betas = betas.dropna()

In [ ]:
betas.loc['Maize']

In [ ]:
betas

In [ ]:
print('Urban, Small HH')
betas['Urban, Small HH'].sort_values(ascending=True)

In [ ]:
print('Urban, Large HH')
betas['Urban, Large HH'].sort_values(ascending=True)

In [ ]:
print('Rural, Small HH')
betas['Rural, Small HH'].sort_values(ascending=True)

In [ ]:
print('Rural, Large HH')
betas['Rural, Large HH'].sort_values(ascending=True)

In [ ]:
w_dict = {'Urban, Small HH' : urban_smallHH.get_w(), 'Urban, Large HH': urban_bigHH.get_w(), 
             'Rural, Small HH' : rural_smallHH.get_w(), 'Rural, Large HH' : rural_bigHH.get_w()}
welfare = pd.DataFrame(w_dict) # made a dataframe with Frisch elasticities for each good in each place
welfare

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [ ]:
# showing distribution of welfare in all households, broken up here by household size

fig, ax = plt.subplots(2,2, figsize = (11,8), dpi=200)


ax[0,0].hist(welfare['Urban, Small HH'], bins=20, density=True, 
            rwidth=0.95, align = 'mid')
ax[0,0].set_title('Urban, Small HH')


ax[0,1].hist(welfare['Urban, Large HH'], bins=20, density=True, 
            rwidth=0.95, align = 'mid')
ax[0,1].set_title('Urban, Large HH')


ax[1,0].hist(welfare['Rural, Small HH'], bins=20, density=True, 
            rwidth=0.95, align = 'mid')
ax[1,0].set_title('Rural, Small HH')


ax[1,1].hist(welfare['Rural, Large HH'], bins=20, density=True, 
            rwidth=0.95, align = 'mid')
ax[1,1].set_title('Rural, Large HH')

plt.savefig('Welfare_Comparison_U-R_HHs.png')

In [ ]:
pbar = p.mean()

In [ ]:
#Finally, define a function to change a single price in the vector $p$:

def my_prices(p0,p=pbar,j=''):
    """
    Change price of jth good to p0, holding other prices fixed.
    """
    p = p.copy()
    p.loc[j] = p0
    return p

def plot_demand(regression, food=''): 
    # set the desired food for the demand plot
    use = food
    # get predicted expenditures
    xhat = regression.predicted_expenditures()
    # Total food expenditures per household
    xbar = xhat.groupby(['i','t','m']).sum()
    # Reference budget
    xref = xbar.quantile(0.5)  # Household at 0.5 quantile is median
    # Reference prices chosen from a particular time; average across place.
    # These are prices per kilogram:
    pbar = p.mean()
    pbar = pbar[regression.beta.index] # Only use prices for goods we can estimate
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    plt.figure(figsize = (5,5))
    # Demand for Food for household at median budget
    plt.plot([regression.demands(xref,my_prices(pbar[use]*s,pbar, j=use))[use] for s in scale],scale, 
            label = 'Median Budget')
    # Demand for Avocado for household at 25% percentile
    plt.plot([regression.demands(xbar.quantile(0.25),my_prices(pbar[use]*s,pbar,j=use))[use] for s in scale],scale, 
            label = '25th Percentile')
    # Demand for Avocado for household at 75% percentile
    plt.plot([regression.demands(xbar.quantile(0.75),my_prices(pbar[use]*s,pbar, j=use))[use] for s in scale],scale, 
            label = '75th Percentile')
    plt.legend(facecolor = 'white')

    plt.ylabel(f"Price (relative to base {pbar[use]:.2f})")
    plt.xlabel(f"Quantity Demanded")
    plt.title(f'{use}', fontsize = 15)

## Plotting Demand for Different Brackets

In [ ]:
def my_prices(p0,p=pbar,j=''):
    """
    Change price of jth good to p0, holding other prices fixed.
    """
    p = p.copy()
    p.loc[j] = p0
    return p

In [ ]:
def plot_demand_comparison_HH(regression1_small, regression2_large, geog = '', food=''): 
    # set the desired food for the demand plot
    use = food
    # get predicted expenditures
    xhat1 = regression1_small.predicted_expenditures()
    xhat2 = regression2_large.predicted_expenditures()
    # Total food expenditures per household
    xbar1 = xhat1.groupby(['i','t','m']).sum()
    xbar2 = xhat2.groupby(['i','t','m']).sum()
    # Reference budget
    xref1 = xbar1.quantile(0.5)  # Household at 0.5 quantile is median
    xref2 = xbar2.quantile(0.5)  # Household at 0.5 quantile is median
    # Reference prices chosen from a particular time; average across place.
    # These are prices per kilogram:
    pbar = p.mean()
    pbar1 = pbar[regression1_small.beta.index] # Only use prices for goods we can estimate
    pbar2 = pbar[regression2_large.beta.index] # Only use prices for goods we can estimate
    
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    plt.figure(figsize = (5,5), dpi=200)
    # Demand for Food for Urban Households at median budget
    plt.plot([regression1_small.demands(xref1,my_prices(pbar1[use]*s,pbar1, j=use))[use] for s in scale],scale, 
            label = f'Small HH, (Elasticity = {regression1_small.get_beta()[use]:.2f})', lw = 2, color = 'skyblue')
    # Demand for Food for Rural Households at median budget
    plt.plot([regression2_large.demands(xref2,my_prices(pbar2[use]*s,pbar2, j=use))[use] for s in scale],scale, 
            label = f'Large HH, (Elasticity = {regression2_large.get_beta()[use]:.2f})', lw = 2, color = 'blue')
    
    plt.legend(facecolor = 'white')

    plt.ylabel(f"Price [relative to base {pbar[use]:.2f}]")
    plt.xlabel(f"Quantity Demanded")
    plt.title(f'{use} Demand, {geog} Households', fontsize = 15)

In [ ]:
plot_demand_comparison_HH(rural_smallHH, rural_bigHH, geog='Rural', food = 'Maize')
plt.savefig('Maize_Demand_Rural.png')

In [ ]:
def plot_demand_comparison_geog(regression1, regression2, hh_size = '', food=''): 
    # set the desired food for the demand plot
    use = food
    # get predicted expenditures
    xhat1 = regression1.predicted_expenditures()
    xhat2 = regression2.predicted_expenditures()
    # Total food expenditures per household
    xbar1 = xhat1.groupby(['i','t','m']).sum()
    xbar2 = xhat2.groupby(['i','t','m']).sum()
    # Reference budget
    xref1 = xbar1.quantile(0.5)  # Household at 0.5 quantile is median
    xref2 = xbar2.quantile(0.5)  # Household at 0.5 quantile is median
    # Reference prices chosen from a particular time; average across place.
    # These are prices per kilogram:
    pbar = p.mean()
    pbar1 = pbar[regression1.beta.index] # Only use prices for goods we can estimate
    pbar2 = pbar[regression2.beta.index] # Only use prices for goods we can estimate
    
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    plt.figure(figsize = (5,5))# dpi=200)

    
    # Demand for Food for Urban Households at median budget
    plt.plot([regression1.demands(xref1,my_prices(pbar1[use]*s,pbar1, j=use))[use] for s in scale],scale, 
            label = f'Urban, (Elasticity = {regression1.get_beta()[use]:.2f})', lw = 2, color = 'dodgerblue')
    # Demand for Food for Rural Households at median budget
    plt.plot([regression2.demands(xref2,my_prices(pbar2[use]*s,pbar2, j=use))[use] for s in scale],scale, 
            label = f'Rural, (Elasticity = {regression2.get_beta()[use]:.2f})', lw = 2, color = 'goldenrod')
    
    plt.legend(facecolor = 'white')

    plt.ylabel(f"Price [relative to base {pbar[use]:.2f}]")
    plt.xlabel(f"Quantity Demanded")
    plt.title(f'{use} Demand, {hh_size} Households', fontsize = 15)

In [ ]:
def plot_demand_comparison_all(urban_big, rural_big, urban_small, rural_small, food=''): 
    # set the desired food for the demand plot
    use = food
    # get predicted expenditures
    xhat_urban_big = urban_big.predicted_expenditures()
    xhat_rural_big = rural_big.predicted_expenditures()
    xhat_urban_small = urban_small.predicted_expenditures()
    xhat_rural_small = rural_small.predicted_expenditures()

    
    # Total food expenditures per household
    xbar1 = xhat_urban_big.groupby(['i','t','m']).sum()
    xbar2 = xhat_rural_big.groupby(['i','t','m']).sum()
    xbar3 = xhat_urban_small.groupby(['i','t','m']).sum()
    xbar4 = xhat_rural_small.groupby(['i','t','m']).sum()
    
    # Reference budget
    xref1 = xbar1.quantile(0.5)  # Household at 0.5 quantile is median
    xref2 = xbar2.quantile(0.5)  # Household at 0.5 quantile is median
    xref3 = xbar3.quantile(0.5)  # Household at 0.5 quantile is median
    xref4 = xbar4.quantile(0.5)  # Household at 0.5 quantile is median
    
    # Reference prices chosen from a particular time; average across place.
    # These are prices per kilogram:
    pbar = p.mean()
    pbar1 = pbar[urban_big.beta.index] # Only use prices for goods we can estimate
    pbar2 = pbar[rural_big.beta.index] # Only use prices for goods we can estimate
    pbar3 = pbar[urban_small.beta.index] # Only use prices for goods we can estimate
    pbar4 = pbar[rural_small.beta.index] # Only use prices for goods we can estimate
    
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    plt.figure(figsize = (12,8))# dpi=200)


    # Demand for Food for Small Urban Households at median budget
    plt.plot([urban_small.demands(xref3,my_prices(pbar3[use]*s,pbar3, j=use))[use] for s in scale],scale, 
            label = f'Urban, HH < 8 (Elasticity = {urban_small.get_beta()[use]:.2f})', lw = 3, color = 'violet')
    
    # Demand for Food for Big Urban Households at median budget
    plt.plot([urban_big.demands(xref1,my_prices(pbar1[use]*s,pbar1, j=use))[use] for s in scale],scale, 
            label = f'Urban, HH > 8 (Elasticity = {urban_big.get_beta()[use]:.2f})', lw = 3, color = 'purple')
    
    # Demand for Food for Small Rural Households at median budget
    plt.plot([rural_small.demands(xref4,my_prices(pbar4[use]*s,pbar4, j=use))[use] for s in scale],scale, 
            label = f'Rural, HH < 8  (Elasticity = {rural_small.get_beta()[use]:.2f})', lw = 3, color = 'skyblue')
    
    # Demand for Food for Big Rural Households at median budget
    plt.plot([rural_big.demands(xref2,my_prices(pbar2[use]*s,pbar2, j=use))[use] for s in scale],scale, 
            label = f'Rural, HH > 8 (Elasticity = {rural_big.get_beta()[use]:.2f})', lw = 3, color = 'blue')
    
    
    plt.legend(facecolor = 'white', fontsize = 12)

    plt.ylabel(f"Price [relative to base {pbar[use]:.2f}]", fontsize = 17)
    plt.xlabel(f"Quantity Demanded", fontsize = 17)
    plt.title(f'{use} Demand', fontsize = 25)

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = 'Ice Cream')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = 'Avocado')
plt.savefig('Avocado_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = 'Chilies')
plt.savefig('Chilies_HH_Overall.png')

In [ ]:
foods = ['Beans', 'Oranges', 'Coffee (instant)', 'Salt', 'Rolled Oats', 'Garlic']

for i in foods: 
    plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = i)
    plt.savefig(f'{i}_Overall.png')
    

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Maize')
plt.savefig('Maize_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Granulated Sugar')
plt.savefig('Sugar_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Corn Tamales')
plt.savefig('Tamales_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Garlic')
plt.savefig('Garlic_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Avocado')
plt.savefig('Avocado_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Pork')
plt.savefig('Pork_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Salt')
plt.savefig('Salt_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Beans')
plt.savefig('Beans_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Oranges')
plt.savefig('Oranges_HH_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food='Rolled Oats')
plt.savefig('Rolled_Oats_HH_Overall.png')

In [ ]:
plot_demand_comparison_geog(urban_bigHH, rural_bigHH, hh_size = 'Large', food = 'Pork')
plot_demand_comparison_geog(urban_smallHH, rural_smallHH, hh_size = 'Small', food = 'Pork')

In [ ]:
# plot_demand_comparison_geog(urban_bigHH, rural_bigHH, hh_size = 'Large', food = 'Avocado')

In [ ]:
betas['$\Delta_E$, Urban'] = np.abs(betas['Urban, Large HH'] - betas['Urban, Small HH'])
betas['$\Delta_E$, Rural'] = np.abs(betas['Rural, Large HH'] - betas['Rural, Small HH'])

betas['$\Delta_E$, Small HH'] = np.abs(betas['Urban, Small HH'] - betas['Rural, Small HH'])
betas['$\Delta_E$, Large HH'] = np.abs(betas['Urban, Large HH'] - betas['Rural, Large HH'])

In [ ]:
betas = betas.dropna()

In [ ]:
#betas = betas.sort_values(by='$\Delta$ (|U-R|)', axis=0, ascending=False)

In [ ]:
sorted_by_largeHH = betas.sort_values(by='$\Delta_E$, Large HH', axis=0, ascending=False)
sorted_by_rural = betas.sort_values(by='$\Delta_E$, Rural', axis=0, ascending=False)

In [ ]:
foods2 = ['Celery', 'Queso', 'Queso', 'Beef', 'Sweets', 'Bananas']

for i in foods2: 
    plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = i)
    plt.savefig(f'{i}_Overall.png')

In [ ]:
plot_demand_comparison_all(urban_bigHH, rural_bigHH, urban_smallHH, rural_smallHH, food = 'Oranges')
plt.savefig('Oranges_Overall.png')

In [ ]:
sorted_by_largeHH

In [ ]:
sorted_by_rural

In [ ]:
big_diff_foods_largeHH = betas[0:15].index

In [ ]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

## Main Comparison Food: Pork (it is high up there for Rural HHs and Maize didn't have data across) 

## From the Differences, Plot the Demand curves for the goods where there's the largest difference between the elaasticities: 

In [ ]:
first5_bigdiff = big_diff_foods_largeHH[0:4] 

for i in big_diff_foods_largeHH: 
    plot_demand_comparison_geog(urban_bigHH, rural_bigHH, hh_size = 'Large', food=i)

In [ ]:
def demand_compare_all4(regression1, regression2, regression1_small, regression2_large, hh_size = '', food=''):
    # set the desired food for the demand plot
    use = food
    # get predicted expenditures
    xhat1 = regression1.predicted_expenditures()
    xhat2 = regression2.predicted_expenditures()
    # Total food expenditures per household
    xbar1 = xhat1.groupby(['i','t','m']).sum()
    xbar2 = xhat2.groupby(['i','t','m']).sum()
    # Reference budget
    xref1 = xbar1.quantile(0.5)  # Household at 0.5 quantile is median
    xref2 = xbar2.quantile(0.5)  # Household at 0.5 quantile is median
    # Reference prices chosen from a particular time; average across place.
    # These are prices per kilogram:
    pbar = p.mean()
    pbar1 = pbar[regression1.beta.index] # Only use prices for goods we can estimate
    pbar2 = pbar[regression2.beta.index] # Only use prices for goods we can estimate
    
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    fig, ax = plt.subplots(2,2, figsize = (10,10), dpi=200)

    
    # Demand for Food for Urban Households at median budget
    ax[0,0].plot([regression1.demands(xref1,my_prices(pbar1[use]*s,pbar1, j=use))[use] for s in scale],scale, 
            label = f'Urban, (Elasticity = {regression1.get_beta()[use]:.2f})', lw = 2, color = 'purple')
    ax[0,0].set_title(f'{use} Demand, Urban Households', fontsize = 15)

    # Demand for Food for Rural Households at median budget
    ax[0,0].plot([regression2.demands(xref2,my_prices(pbar2[use]*s,pbar2, j=use))[use] for s in scale],scale, 
            label = f'Rural, (Elasticity = {regression2.get_beta()[use]:.2f})', lw = 2, color = 'red')
    ax[0,1].set_title(f'{use} Demand, Rural Households', fontsize = 15)
    
    # Demand for Food for Urban Households at median budget
    ax[1,0].plot([regression1_small.demands(xref1,my_prices(pbar1[use]*s,pbar1, j=use))[use] for s in scale],scale, 
            label = f'Small HH, (Elasticity = {regression1_small.get_beta()[use]:.2f})', lw = 2, color = 'dodgerblue')
    # Demand for Food for Rural Households at median budget
    ax[1,1].plot([regression2_large.demands(xref2,my_prices(pbar2[use]*s,pbar2, j=use))[use] for s in scale],scale, 
            label = f'Large HH, (Elasticity = {regression2_large.get_beta()[use]:.2f})', lw = 2, color = 'goldenrod')
    
    plt.legend(facecolor = 'white')
    plt.setp(ax, xlabel="Quantity Demanded", ylabel=f"Price [relative to base {pbar[use]:.2f}]")
    plt.ylabel(f"Price [relative to base {pbar[use]:.2f}]")
    plt.xlabel(f"Quantity Demanded")
    plt.title(f'{use} Demand, {geog} Households', fontsize = 15)
    
    plt.legend(facecolor = 'white')

    plt.ylabel(f"Price [relative to base {pbar[use]:.2f}]")
    plt.xlabel(f"Quantity Demanded")
    plt.title(f'{use} Demand, {hh_size} Households', fontsize = 15)


In [ ]:
top_3 = ['Pork', 'Salt', 'Beans']

# Pork Demand Curve Comparisons

In [ ]:
demand_compare_all_4(food = 'Pork')

# Salt Demand Curve Comparisons

In [ ]:
demand_compare_all_4(food = 'Salt')

# Beans Demand Curve Comparison

In [ ]:
demand_compare_all_4(food = 'Beans')

# Avocado Demand Curve Comparison

In [ ]:
demand_compare_all_4(food = 'Avocado')

In [ ]:
import plotly.express as px
px.scatter(x_Rural)

In [ ]:
px.scatter(x_Urban)

In [ ]:
px.bar(x_Rural, x=food_columns)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(x_Rural[food_columns])

In [ ]:
food_columns = ['Avocado', 'Bananas', 'Beans', 'Beef', 'Beef Entrails', 'Beer', 'Beets',
       'Bread (french)', 'Bread (sliced)', 'Bread (sweet)', 'Brown Sugar',
       'Butter', 'Cabbages', 'Cakes', 'Candies', 'Carrots', 'Celery',
       'Chayote', 'Chicken', 'Chicken Giblets', 'Chilies', 'Chocolate',
       'Cigarettes', 'Coffee (instant)', 'Cookies', 'Cooking Oil',
       'Corn Atole', 'Corn Dough (fresh(', 'Corn Flakes', 'Corn Tamales',
       'Cornmeal', 'Crema Fresca', 'Cucumbers', 'Dried Fruit', 'Dried Seeds',
       'Eggs', 'Fish (Canned)', 'Fish (Fresh)', 'Garlic', 'Granulated Sugar',
       'Herbs', 'Honey, Molasses', 'Ice Cream', 'Incaparina', 'Infant Formula',
       'Jam', 'Juice (Branded)', 'Juices (Packaged)', 'Lard',
       'Leafcutter Ants and Other Insect', 'Lemons', 'Lettuce', 'Liquor',
       'Maize', 'Mangos', 'Margarine', 'Melons', 'Milk', 'Milk (condensed)',
       'Milk (powdered)', 'Mineral Water', 'Mushrooms', 'Onions', 'Oranges',
       'Other', 'Other Atoles', 'Other Canned Goods', 'Other Sauces', 'Paches',
       'Papaya', 'Pasta', 'Peas', 'Pineapples', 'Pork', 'Pork (cracklings)',
       'Potatoes', 'Pumpkin', 'Queso', 'Rice', 'Rolled Oats', 'Rural', 'Salt',
       'Sausages', 'Soup', 'Spices', 'Sweets', 'Tea', 'Tomato Sauce',
       'Tomatoes', 'Tortillas', 'Tostadas', 'Vegetable Oil', 'Water (bottle)',
       'Watermelon', 'Wheat Flour', 'Yogurt', 'Yucca'